In [ ]:
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(12,8)})
%matplotlib inline
import glob
import csv
import os
import tarfile
import math
import numpy as np 
from tqdm import tqdm
import json
from progressive.bar import Bar
import pickle as pkl
import networkx as nx
from __future__ import unicode_literals

#Make sure to use python 2 kernerls since the pickles are incompatible with python 3 :( 

In [ ]:
# import dill

In [ ]:

# dill._dill._reverse_typemap['ObjectType'] = object

In [ ]:
# with open('../mappings/Bipartite_Drug_graph.pkl','rb') as f:
#     drug_association_graph  = pkl.load(f)
    
# with open("../mappings/Bipartite_Drug_graph.pkl", 'rb') as f:
#     drug_association_graph = dill.load(f, encoding="bytes") 

# with open('../mappings/Bipartite_Drug_category_graph.pkl','rb') as f:
#     drug_cat_association_graph  = pkl.load(f)
    
# with open("../mappings/Bipartite_Drug_category_graph.pkl", 'rb') as f:
#     drug_cat_association_graph = pkl.load(f, encoding="bytes")
mappings_dir = '../mappings/'
drug_association_graph = nx.read_gexf(mappings_dir + 'drug_association_graph.gexf')
drug_cat_association_graph = nx.read_gexf(mappings_dir + 'category_association_graph.gexf')

In [ ]:
print (len(drug_association_graph.edges()))
print (len(drug_cat_association_graph.edges()))
print (len(drug_association_graph.nodes()) )
print (len(drug_cat_association_graph.nodes()))

In [ ]:
#Old files till june 2019
# files = glob.glob('../../BL_Work/openPrescribe/uncompressed/*.tar.gz')
# print(files)

In [ ]:
#NEw files after July 2019

files = glob.glob('/10TBDrive_2/sagar/NHS_data/EPD*.zip')
print(files)

In [ ]:
#Different functions to extract different data form the prescription 

import inspect
import re

def extractPostCodesDict(addrDf):
    postcodeDict = {}
    for index,row in addrDf.iterrows():
        try:
            postcodeDict[row[1]] = row[7].strip()
        except:
            print("Found invalid entry")
    return postcodeDict

def checkIndex(index):
    if index%100 == 0:
        return True
    else:
        return False

def getPC(key , postcodeDict):
    codes = []
    for k in key:
        if k in postcodeDict:
            codes.append(postcodeDict[k])
        else:
            codes.append('')
    return pd.Series(codes,index=key.index)

def getPostcode(df,postcodeDict):
    df[10] = ''
    df[10] = df.groupby(2)[2].apply(getPC,postcodeDict)   
    return df

def getDrugFamily(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = dcode
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDisease(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = diseaseMap[dcode]['disease'].replace('\"','').replace('+',' ')
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDrug(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = diseaseMap[dcode]['chemName']
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDrugPotency(key):
    name = list(set(key))
    if len(name) > 1:
        print("found synonyms")
    text= name[-1]
    found = 0.0
    switch1 = text.find('mcg')
    switch2 = text.find('mg')
    switch3 = text.find('ml')
    
    if switch1 > 0 or switch2 > 0 or switch3 > 0:
        weight = re.findall(r'[0-9]*\.?[0-9]+', text)
        if len(weight) > 0:
            found = max([float(k) for k in weight])
            if switch1 > 0:
                found = found/1000.0
    potency = [found]*len(key)
    return pd.Series(potency,index=key.index)

def countSpecificDrugs(Df, drugs,GPs):
    df_slice = Df.groupby(3)[3].apply(getDrug,drugs)
    selected = df_slice[df_slice!='N/A']
    len(selected)
    df_selected =  Df.iloc[selected.index,:]
    df_selected = df_selected[df_selected[2].isin(GPs.keys())]
    return np.sum(df_selected[5])


def countSpecificDrugCosts(Df, drugs,GPs):
    df_slice = Df.groupby(3)[3].apply(getDrug,drugs)
    selected = df_slice[df_slice!='N/A']
    len(selected)
    df_selected =  Df.iloc[selected.index,:]
    df_selected = df_selected[df_selected[2].isin(GPs.keys())]
    return np.sum(df_selected[6])

def countDrugsByCategoryList(pdp,codes):
    total_drugs = 0.0
    drugs = None
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[5])
    return total_drugs

def countPrescriptionsByCategoryList(pdp,codes):
    total_drugs = 0.0
    drugs = None
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=len(group)
    return total_drugs


def countDrugsCostByCategoryList(pdp,codes):
    total_drugs = 0.0
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[7])
    return total_drugs

def countDrugsCostByGenerics(pdp,codes):
    total_drugs = 0.0
    generics= pdp[pdp[20] == 'AA']
    for name , group in generics.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[7])
    return total_drugs

def compareCostsForGenericsAndBranded(pdp,codes):
    genericsCosts = {}
    brandedCosts = {}
    for name , group in pdp.groupby(16):
        for dcode in codes:
            if name == dcode:
                generics= group[group[20] == 'AA']
                if len(generics)>0:
                    total_drugs =np.sum(group[7])
                    generic_drugs = np.sum(generics[7])
                    brandedCosts[dcode] = total_drugs - generic_drugs
                    genericsCosts[dcode] = generic_drugs
                else:
                    print("Did not find any generic drugs")
                    continue
    return brandedCosts , genericsCosts

def func_Cost(potgroup):
    
    generics= potgroup.loc[potgroup[20] == 'AA']
    nonGenerics =  potgroup.loc[potgroup[20] != 'AA']

    minCost = np.min(generics[7])
    minpotdf = generics.loc[generics[7] == minCost]
    minQuant = np.min(minpotdf[8])
    if minQuant == 0:
        normalizer = minCost
    else:
        normalizer = float(minCost)/float(minQuant)
    
    if np.isnan(normalizer):
        normalizer = 1.0
    potgroup[21] = normalizer
    
    minCostBrand = np.min(nonGenerics[7])
    minpotdfBrand = nonGenerics.loc[nonGenerics[7] == minCostBrand]
    minQuantBrand = np.min(minpotdfBrand[8])
    
    if minQuantBrand == 0:
        unitNonGenericCost = minCostBrand
    else:
        unitNonGenericCost = float(minCostBrand)/float(minQuantBrand)
    
    if np.isnan(unitNonGenericCost):
        unitNonGenericCost = 1.0
    potgroup[22] = unitNonGenericCost
    if unitNonGenericCost > normalizer:
        potgroup[23] = float(unitNonGenericCost - normalizer)*potgroup[8]    
    return potgroup

def func_Drugs(group,codes):
    return group.groupby(15,as_index=False).apply(lambda df : func_Cost(df))
    

def computeSavingsNew(pdp,codes):
    pdp[21] = 0.0
    pdp[22] = 0.0
    pdp[23] = 0.0
    return pdp.groupby(16,as_index=False).apply(lambda df: func_Drugs(df , codes))


def computeSavings(pdp,codes):
    pdp[21] = 0.0
    pdp[22] = 0.0
    pdp[23] = 0.0
    for name , group in pdp.groupby(16):
        #we can remove this to allow computing savings across all drugs
        if name in codes:
            for pot , potgroup in group.groupby(15):
                generics= potgroup.loc[potgroup[20] == 'AA']
                nonGenerics =  potgroup.loc[potgroup[20] != 'AA']

                minCost = np.min(generics[7])
                minpotdf = generics.loc[generics[7] == minCost]
                minQuant = np.min(minpotdf[8])
                normalizer = float(minCost)/float(minQuant)
                potgroup[21] = normalizer

                minCostBrand = np.min(nonGenerics[7])
                minpotdfBrand = nonGenerics.loc[nonGenerics[7] == minCostBrand]
                minQuantBrand = np.min(minpotdfBrand[8])

                unitNonGenericCost = float(minCostBrand)/float(minQuantBrand)
                potgroup[22] = unitNonGenericCost
                
                if unitNonGenericCost > normalizer:
                    nonGenerics[23] = float(unitNonGenericCost - normalizer)*nonGenerics[8]


def countTotalDrugDosage(pdp,codes):
    total_drugs = 0.0
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[19])
    return total_drugs

def normalizePills(keys):
    minima = np.min(keys)
    if minima > 0:
        potency = keys/minima
    else:
        potency = 1.0
    return pd.Series(potency,index=keys.index)
        
    
def normalizePotency(keys):
    minima = np.min(keys)
    if minima > 0:
        potency = keys/minima
    else:
        potency = 1.0
    return pd.Series(potency,index=keys.index)
    
def normalize(dataFrame):
    dataFrame[16] = dataFrame[3].str[:9]
    dataFrame[17] = dataFrame.groupby(16)[8].apply(normalizePills)
    dataFrame[18] = dataFrame.groupby(16)[15].apply(normalizePotency) 
    dataFrame[19] = dataFrame[18]*dataFrame[17]
    dataFrame[20] = dataFrame[3].str[9:11]
    dataFrame[21] = dataFrame[15]/dataFrame[8]
    
    
def normalize_new(dataFrame):
    dataFrame[16] = dataFrame['BNF_CODE'].str[:9]
    dataFrame[17] = dataFrame.groupby(16)['TOTAL_QUANTITY'].apply(normalizePills)
    dataFrame[18] = dataFrame.groupby(16)[15].apply(normalizePotency) 
    dataFrame[19] = dataFrame[18]*dataFrame[17]
    dataFrame[20] = dataFrame['BNF_CODE'].str[9:11]
    dataFrame[21] = dataFrame[15]/dataFrame['TOTAL_QUANTITY']
    
    
    
def doImportantMappings(Df):
    #BNF family
#     Df[11] = ''  
#     Df[11] = Df.groupby(3)[3].apply(getDrugFamily,symptomMap)
#     #Disease
#     Df[12] = ''  
#     Df[12] = Df.groupby(3)[3].apply(getDisease,diseaseMap)
#     #Symptom
#     Df[13] = ''  
#     Df[13] = Df.groupby(3)[3].apply(getDisease,symptomMap)
#     #Checm Name
#     Df[14] = ''  
#     Df[14] = Df.groupby(3)[3].apply(getDrug,symptomMap)
    #Chem potency
    Df[15] = ''  
    Df[15] = Df.groupby(3)[4].apply(getDrugPotency)

    normalize(Df)

    return Df



def doImportantMappings_new(Df):
    #BNF family
#     Df[11] = ''  
#     Df[11] = Df.groupby('BNF_CODE')['BNF_CODE'].apply(getDrugFamily,symptomMap)
#     #Disease
#     Df[12] = ''  
#     Df[12] = Df.groupby('BNF_CODE')['BNF_CODE'].apply(getDisease,diseaseMap)
#     #Symptom
#     Df[13] = ''  
#     Df[13] = Df.groupby('BNF_CODE')['BNF_CODE'].apply(getDisease,symptomMap)
#     #Checm Name
#     Df[14] = ''  
#     Df[14] = Df.groupby('BNF_CODE')['BNF_CODE'].apply(getDrug,symptomMap)
    #Chem potency
    Df[15] = ''  
    Df[15] = Df.groupby('BNF_CODE')['BNF_DESCRIPTION'].apply(getDrugPotency)

    normalize_new(Df)

    return Df



In [ ]:
# chem = pd.read_csv('../mappings/CHEM_MASTER_MAP.csv')
# len(chem)

# # chem = chem.dropna()

# matched = chem[chem['UNII_drugbank']!='[]']

# matchedMap = {}
# for index,row in matched.iterrows():
#     if row['UNII_drugbank'] not in matchedMap:
#          matchedMap[row['UNII_drugbank']] = []
#     matchedMap[row['UNII_drugbank']].append(row['BNF_code'])

In [ ]:
monthListAll = []
for f in files: 
    weight = re.findall(r'[0-9]*\.?[0-9]+', f)
    monthListAll.append(weight[-1])
#     print(weight)
monthListAll = list(set(monthListAll))
monthListAll.sort()
print(monthListAll)

In [ ]:
#Old mapping
# GP_META = pd.read_csv('mappings/epraccur.csv',header=None)
# GP_META.rename(columns={0:'BP_code',
#                         1:'Name',
#                         2:'Grouping',
#                         3:'National_geo',
#                         4:'Addr1',
#                         5:'Addr2',
#                         6:'Addr3',
#                         7: 'Addr4',
#                         8:'Addr5',
#                         9:'Postcode',
#                         10:'Open',
#                         11:'Closed',
#                         12:'Status',
#                         13:'Org type code',
#                         14:'Commissioner',
#                         15:'Join provider',
#                         16:'Left provider',
#                         17:'Contact',
#                         18:'Null1',
#                         19:'Null2',
#                         20:'Null3',
#                         21:'Amended',
#                         22:'Null4',
#                         23:'Provider',
#                         24:'Null5',
#                         25:'Setting',
#                         26:'Null6'}, 
#                  inplace=True)

In [ ]:
#New GP file
GP_META = pd.read_csv('mappings/epraccur_2021.csv',header=None)
GP_META.rename(columns={0:'BP_code',
                        1:'Name',
                        2:'Grouping',
                        3:'National_geo',
                        4:'Addr1',
                        5:'Addr2',
                        6:'Addr3',
                        7: 'Addr4',
                        8:'Addr5',
                        9:'Postcode',
                        10:'Open',
                        11:'Closed',
                        12:'Status',
                        13:'Org type code',
                        14:'Commissioner',
                        15:'Join provider',
                        16:'Left provider',
                        17:'Contact',
                        18:'Null1',
                        19:'Null2',
                        20:'Null3',
                        21:'Amended',
                        22:'Null4',
                        23:'Provider',
                        24:'Null5',
                        25:'Setting',
                        26:'Null6'}, 
                 inplace=True)

In [ ]:
Open_filtered_GPs = {}
for index,row in GP_META.iterrows():
    if row ['Setting'] == 4 and row['Status'] =='A':
        Open_filtered_GPs[row['BP_code']] = row['Postcode'].strip()

In [ ]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='Serializing.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [ ]:
# pdp_old = pd.read_csv('../../BL_Work/openPrescribe/uncompressed/T201206PDPI BNFT.csv.tar.gz',compression='infer',header=None,skiprows=1)

In [ ]:
# pdp_old.head()

In [ ]:
pdp = pd.read_csv('/10TBDrive_2/sagar/NHS_data/'+'EPD_'+str(monthListAll[0]) +'.zip')

In [ ]:
# pdp[1000:1009]

In [ ]:
# print set(pdp['BNF_CHAPTER_PLUS_CODE'])

In [ ]:
# cat_filters = ['15: Anaesthesia', '20: Dressings', '23: Stoma Appliances', '21: Appliances','18: Preparations used in Diagnosis']

In [ ]:
pdp = pdp.dropna()
pdp_filtered = pdp[pdp['PRACTICE_CODE'].isin(Open_filtered_GPs.keys())].copy(deep=True)
# pdp_filtered = pdp[~pdp['BNF_CHAPTER_PLUS_CODE'].isin(cat_filters)]

In [ ]:
len(pdp) , len(pdp_filtered)

In [ ]:
pdp_filtered = doImportantMappings_new(pdp_filtered)

In [ ]:
# print(pdp_filtered.iloc[51000])

In [ ]:
print (pdp_filtered[pdp_filtered[16]=='0403040X0'].iloc[0])

In [ ]:
# USE this for pre 2019-06 data
# monthList = monthListAll
# # monthList = [u'201601', u'201602', u'201603',]
# for month in tqdm(monthList):
#     filename = '../BL_Work/openPrescribe/serialized/' + month + '.gz'
#     if not os.path.exists(filename):
#         logging.info("working with File: "+str(filename))
#         pdp = readPrescriptions(month,files)
#         pdp = pdp.dropna()
#         pdp = getPostcode(pdp , postcodes)
#         pdp_filtered = pdp[pdp[2].isin(Open_filtered_GPs.keys())].copy(deep=True)
#         pdp_filtered = doImportantMappings(pdp_filtered,DiseaseDrugs,symptoms_matched)
#         pdp_filtered.to_csv(filename,compression='gzip')

In [ ]:
# # USE this for post 2019-06 data
monthList = monthListAll
# monthList = [u'201601', u'201602', u'201603',]
for month in tqdm(monthList):
    filename = '/10TBDrive_2/sagar/NHS_data/serialized/' + month + '.gz'
    logging.info("working with File: "+str(filename))
    pdp = pd.read_csv('/10TBDrive_2/sagar/NHS_data/'+'EPD_'+str(month) +'.zip')
    pdp = pdp.dropna()
    pdp_filtered = pdp[pdp['PRACTICE_CODE'].isin(Open_filtered_GPs.keys())].copy(deep=True)
#         pdp_filtered = pdp[~pdp['BNF_CHAPTER_PLUS_CODE'].isin(cat_filters)]
    pdp_filtered = doImportantMappings_new(pdp_filtered)
    pdp_filtered.to_csv(filename,compression='gzip')
    del pdp_filtered
    del pdp

In [ ]:
# from itertools import izip_longest
# def grouper(iterable, n, fillvalue=None):
#     # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
#     args = [iter(iterable)] * n
#     return izip_longest(fillvalue=fillvalue, *args)

# for months in grouper(monthListAll, 5):
#     print months
                        